In [1]:
!pip install -q transformers datasets sentencepiece
# sentencepiece installation will require restarting kernel after installation to take effect

In [2]:
import sentencepiece
print(sentencepiece.__version__)


0.2.0


In [3]:
#!pip install transformers --upgrade

In [4]:
from transformers import (
    MT5ForConditionalGeneration,
    MT5Tokenizer,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)
from datasets import load_dataset, Dataset
import numpy as np
import torch
import re
from transformers import AutoTokenizer

# Load mT5 Model and Tokenizer
# ===============================
model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")
tokenizer = MT5Tokenizer.from_pretrained("google/mt5-small")
MAX_LEN = 64

Error during conversion: ChunkedEncodingError(ProtocolError("Connection broken: InvalidChunkLength(got length b'', 0 bytes read)", InvalidChunkLength(got length b'', 0 bytes read)))
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'MT5Tokenizer'.
You are using the default legacy behaviour of the <class 'transformers.models.mt5.tokenization_mt5.MT5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [5]:
# ===============================
# Load and Clean Sanskrit Dataset
# ===============================
# Load and preprocess the dataset
dataset = load_dataset('oscar', 'unshuffled_deduplicated_hi', split='train[:5%]')

In [6]:

def clean_sanskrit_text(example):
    text = example["text"]
    
    # Remove zero-width characters and extra spaces
    text = re.sub(r'[\u200b-\u200d]', '', text)         # Remove zero-width characters
    text = re.sub(r'\s+', ' ', text)                    # Collapse multiple spaces/newlines
    text = text.strip()                                 # Trim leading/trailing whitespace
     # Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    # Remove non-Sanskrit characters (retain Devanagari script)
    text = re.sub(r'[^\u0900-\u097F\s]', '', text)
    
    return {"text": text}

dataset = dataset.map(clean_sanskrit_text)

def filter_by_word_count(example):
    word_count = len(example['text'].split())
    return 15 < word_count < 50

dataset = dataset.filter(filter_by_word_count)
print(dataset[0])

Filter:   0%|          | 0/95469 [00:00<?, ? examples/s]

{'id': 6, 'text': 'आप अश्लील वीडियो देख रहे हैं देसी बड़े स्तन बंगाली गृहिणीबंगाली अश्लील। यह वीडियो  सबसे बड़ी अश्लील वेबसाइट भारतीय सेक्स पर पोस्ट किया गया है। वीडियो की लंबाई   के बारे में   है। आप खोज बॉक्स का उपयोग करके अधिक गर्म अश्लील वीडियो खोज सकते हैं। मज़े करो  '}


In [7]:
import transformers
print(transformers.__version__)
print(transformers.__file__)


4.51.3
/opt/conda/lib/python3.10/site-packages/transformers/__init__.py


In [8]:
len(dataset)

15673

In [9]:
dataset[1]

{'id': 10,
 'text': ' प्रथम द्वितीय तृतीय चतुर्थी पंचमी सृष्टि सप्तमी अस्टमी नवमी दसमी एकादसी द्वादसी त्रयोदसी चतुर्दसी पंचदसी पूर्णिमा अमावस्या'}

In [10]:
dataset

Dataset({
    features: ['id', 'text'],
    num_rows: 15673
})

In [11]:
dataset[46]

{'id': 245,
 'text': 'कुंवर सीपी सिंह युवा टीवी पत्रकार कुंवर सीपी सिंह के बारे में सूचना मिल रही है कि उनका निधन हो गया बताया जाता है कि वे रात में सोए तो'}

In [12]:
from transformers import Seq2SeqTrainingArguments

# 5. Tokenize the Dataset
# ===============================
max_input_length = MAX_LEN
max_target_length = MAX_LEN

def preprocess(example):
    input_text = example["text"]
    input_ids = tokenizer(
        input_text,
        padding="max_length",
        truncation=True,
        max_length=max_input_length
    )

    labels = tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=max_target_length
    )

    input_ids["labels"] = labels["input_ids"]
    return input_ids

tokenized_dataset = dataset.map(preprocess, batched=True, remove_columns=["text"])


Map:   0%|          | 0/15673 [00:00<?, ? examples/s]

In [13]:
#tokenized_dataset[0]

In [14]:
def preprocess_completion(examples):
    new_examples = {"input": [], "target": []}

    for text in examples["text"]:
        words = text.strip().split()
        # Split roughly in the middle to create a meaningful prompt-completion pair
        split_point = len(words) // 2
        input_part = " ".join(words[:split_point])
        target_part = " ".join(words[split_point:])
        
        #print(input_part, " ---- ", target_part)
        
        new_examples["input"].append(input_part)
        new_examples["target"].append(target_part)

    # Tokenize input
    model_inputs = tokenizer(
        new_examples["input"],
        padding="max_length",
        truncation=True,
        max_length=max_input_length,
        return_attention_mask=True,
        add_special_tokens=True,
    )

    # Tokenize target
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            new_examples["target"],
            padding="max_length",
            truncation=True,
            max_length=max_target_length,
            return_attention_mask=True,
            add_special_tokens=False,
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [15]:
from transformers import Seq2SeqTrainingArguments  # not TrainingArguments

def get_training_args(output_dir="./outputs"):
    return Seq2SeqTrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=32,
        num_train_epochs=1,
        learning_rate=5e-4,
        logging_dir="./logs",
        logging_steps=1000,
        save_steps=1000,
        save_total_limit=1,
        predict_with_generate=True,
        fp16=torch.cuda.is_available()
    )


# Trainer setup
def train_model(preprocess_fn, output_dir, remove_cols):
    tokenized_dataset = dataset.map(preprocess_fn, batched=True, remove_columns=remove_cols)
    training_args = get_training_args(output_dir)
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
        tokenizer=tokenizer,
        data_collator=DataCollatorForSeq2Seq(tokenizer, model=model)
    )
    trainer.train()

# Task-specific training functions
def train_completion():
    print("Training for text completion...")
    train_model(preprocess_completion, "./mt5-sanskrit-completion", remove_cols=["text"])


In [16]:
train_completion()

Training for text completion...


Map:   0%|          | 0/15673 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/var/tmp/ipykernel_6861/788856752.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past

Step,Training Loss


In [17]:
'''
from transformers import (
    MT5ForConditionalGeneration,
    MT5Tokenizer,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)
from datasets import load_dataset, Dataset
import numpy as np
import torch
import re
from transformers import AutoTokenizer

checkpoint_dir = "mt5-sanskrit-completion/checkpoint-5000/"  # your saved checkpoint directory

tokenizer = AutoTokenizer.from_pretrained(checkpoint_dir)
model = MT5ForConditionalGeneration.from_pretrained(checkpoint_dir)
'''

'\nfrom transformers import (\n    MT5ForConditionalGeneration,\n    MT5Tokenizer,\n    Seq2SeqTrainer,\n    Seq2SeqTrainingArguments,\n    DataCollatorForSeq2Seq\n)\nfrom datasets import load_dataset, Dataset\nimport numpy as np\nimport torch\nimport re\nfrom transformers import AutoTokenizer\n\ncheckpoint_dir = "mt5-sanskrit-completion/checkpoint-5000/"  # your saved checkpoint directory\n\ntokenizer = AutoTokenizer.from_pretrained(checkpoint_dir)\nmodel = MT5ForConditionalGeneration.from_pretrained(checkpoint_dir)\n'

In [18]:
# Evaluation functions
def evaluate(prompt):
    input_text = prompt
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(model.device)
    outputs = model.generate(input_ids=input_ids, max_length=MAX_LEN)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("\nInput:", prompt)
    print("Output:", result)
    #print(outputs, outputs[0])

# fine-tuning not sufficient - to generate meaningful sentences - will take hundreds of thousands of lines of text and many epochs, which is not achievable with current resources

In [19]:

evaluate("प्रथम द्वितीय तृतीय चतुर्थी पंचमी सृष्टि सप्तमी")


Input: प्रथम द्वितीय तृतीय चतुर्थी पंचमी सृष्टि सप्तमी
Output: के रई हमले


In [20]:
#'दीवाली इस साल नवंबर को बुधवार को पड रही है। इस दिन माता लक्ष्मी भगवान गणेश व कुबेर जी की पुजा'
evaluate("दीवाली इस साल नवंबर को बुधवार को पड रही है")


Input: दीवाली इस साल नवंबर को बुधवार को पड रही है
Output: के और


In [21]:
evaluate("आज सुबह जब मैं पार्क गया, तो मैंने देखा कि")


Input: आज सुबह जब मैं पार्क गया, तो मैंने देखा कि
Output: करते हुस ही और जब जब उन्हाँ ही पास ही जाने जब और साथ पहचान जब कुछ और साथ बातों


In [22]:
evaluate("वह बच्चा स्कूल से लौटते समय")


Input: वह बच्चा स्कूल से लौटते समय
Output: में लौट लौट दैदिक दैर्ध्य रहा था। वह और साल वह लम्बी वह हालीम और लौटकर और लौते हैं।


In [23]:
evaluate("बारिश इतनी तेज हो रही थी कि लोग")


Input: बारिश इतनी तेज हो रही थी कि लोग
Output: और उसकी जाने वाले ही दैवी और लिए ही थी और और थी ।


In [24]:
evaluate("बचपन की सबसे प्यारी याद मेरे लिए")


Input: बचपन की सबसे प्यारी याद मेरे लिए
Output: <extra_id_0> वह हमेशा याद दिया है। उस थी और हम सब जाने और तुम जाने और इस बात और उस ही हम उन्हें और बाद जाने जाने और जाने जाने और


In [25]:
evaluate("वनवास के समय श्रीराम ने माता सीता और लक्ष्मण के साथ मिलकर")


Input: वनवास के समय श्रीराम ने माता सीता और लक्ष्मण के साथ मिलकर
Output: ही पसली और उन्हें और और


In [26]:
#!rm -r logs 